# Evoked EPSCs and IPSCs

#### **Basic instructions:**
#### 1. **To run code cells**, you can either hit the play button to the left of the cell, or you can select the cell and **press shift-enter**.

#### 2. **The first time** you run this code notebook, you might get a popup asking to choose which version of Python to use (the python "kernel"). **Just hit enter** to choose the base/default version.

#### 3. Make sure you data (.abf) files are in the "data" folder here on the left. You can just copy/paste the files over from where they are saved on your computer.

In [ ]:
%matplotlib widget
import numpy as np
import matplotlib.pyplot as plt
from utils import *
update_plot_defaults()
# %load_ext autoreload
# %autoreload 2

## 1. Choose the data file you want to analyze

#### Put the .abf files with your Ih recordings in the "data/Evoked_PSCs" folder

In [ ]:
data_folder = "data/7-Evoked_PSCs"

from glob import glob
data_files = glob(data_folder+"/*.abf")
print(data_folder)
data_files

Choose which file you want to analyze (copy name from above) and paste the file name here:

In [ ]:
# data_file = 'data/7-Evoked_PSCs/2025_06_13_0008.abf'
# data_file = 'data/7-Evoked_PSCs/2025_06_13_0012.abf'
data_file = 'data/7-Evoked_PSCs/2025_06_13_0018.abf'

Now we can load the file and plot the raw data:

In [ ]:
traces = Trace.from_axon_file(filename=data_file, 
                              load_voltage=True, 
                              load_ttl=True,
                              units=['pA', 'mV', 'V'], 
                              concatenate_sweeps=False)
print(traces)

traces.plot(plot_ttl=False, plot_voltage=False, sweep='all', time_units='ms')
# ax1, ax2 = traces.plot(plot_voltage=True, sweep='all')
plt.show()

## 2. Signal processing

### Optional: apply additional highpass/lowpass filtering

Depending in you recording, you may have 50/60 Hz line noise, high-frequency noise, or drift in your recordings.

The goal here is to only remove the noise with minimal distortion of the data, so be careful not to overdo it

In [ ]:
# Change this to True if you want to subtract the baseline from the sweeps.
additional_filtering = True

In [ ]:
if additional_filtering:
    filtered_traces = traces
    # Step 1: Detrend the data to remove linear or constant trends.
    # filtered_traces = filtered_traces.detrend(detrend_type='linear', num_segments=1)

    # Step 2: Apply a highpass filter to remove low-frequency noise + lowpass to remove high-frequency noise
    filtered_traces = filtered_traces.filter(
        line_freq=60,    # Frequency (Hz) of electrical noise to remove: 50 Hz (in Europe) or 60 Hz (in the US).
        width=1,         # Width (Hz) controls the width of frequency bands around the line frequency the filter cuts out.
        highpass=None,   # Removes low-frequency drift. Set a value in Hz (e.g. 1 for 1 Hz).
        lowpass=2000,    # Removes high-frequency noise. Set a value in Hz (e.g. 100 for 100 Hz).
        order=4)         # Controls sharpness of the filter. Higher = sharper cutoff.


    ax = filtered_traces.plot(plot_voltage=False, plot_ttl=False, sweep='all')
    ax.set_title('After filtering', y=0.98)
    plt.show()

Once you are happy with the filter setting, run the next cell to implement them:

In [ ]:
if additional_filtering:
    traces=filtered_traces

### Optional: apply baseline correction

If your baseline current is not zero, you may need to correct for that to get accurate measurements.

Uncomment this cell block (highlight everything, then cmd+/ on macOS or ctrl+/ on Windows) to run it.

In [ ]:
# Change this to True if you want to subtract the baseline from the sweeps.
subtract_baseline = True

In [ ]:
if subtract_baseline:
    traces.subtract_baseline(start_time = 0, 
                             end_time = 10, 
                             time_units = 'ms',  # specify seconds (s), or milliseconds (ms)
                             channel = 'all')  # Options: 'current', 'voltage', 'all'
    ax1 = traces.plot(plot_voltage=False, plot_ttl=False, time_units='ms', sweep='all')
    ax1.set_title('After baseline subtraction', y=0.98)
    plt.show()
else:
    print("BASELINE NOT SUBTRACTED")


## 3. Measure EPSCs/IPSCs to get paired pulse ratio

(If you want to save images, click on the save icon on the left when you hover over the image)

### 3.1. Get the single-stim response


Here we will determine the event times based on where the stim artifact (capacitive transient) is

In [ ]:
# Find positive current peaks above threshold
stim_times = traces.get_event_times(threshold=200, 
                                    polarity='positive', 
                                    time_units='ms', 
                                    channel='current', sweep=0)
print(f"Stim times: {stim_times} ms")

In [ ]:
window_size = 100 #ms
single_stim_traces = traces.crop(timepoint=stim_times[0], window=window_size, time_units='ms')
ax = single_stim_traces.plot(plot_voltage=False, plot_ttl=False, time_units='ms', sweep='all')
ax.set_title('Single stim',y=0.95)
plt.show()

#### Optional: remove bad traces

In [ ]:
remove_spikes = False

# ---------------------------------------------------------------------------------------------------------
if remove_spikes:
    # Remove sweeps with positive current spikes above or below threshold
    info = single_stim_traces.remove_sweeps_with_spikes(threshold=-2000, polarity='negative', 
                                                channel='current', return_info=True)

    ax = single_stim_traces.plot(plot_voltage=False, plot_ttl=False, time_units='ms', sweep='all')
    # ax.set_ylim(bottom=-600, top=100)
    ax.set_title('Single stim (spikes removed)',y=0.95)
    plt.show()

#### Measure single-stim amplitudes

In [ ]:
single_stim_currents, _ = single_stim_traces.get_measurements(start_time=2, 
                                                            end_time=15, 
                                                            measurement_type='min', 
                                                            time_units='ms')
fig,ax = plt.subplots(figsize=(8,3))
ax.plot(single_stim_currents, '-o')
ax.set_xticks(range(len(single_stim_currents)))
ax.set_ylabel('single-stim currents (pA)')
ax.set_xlabel("sweep nr.")
plt.show()

### 3.2. Get the response to the first paired-pulse stim

In [ ]:
first_pulse_traces = traces.crop(timepoint=stim_times[1], window=window_size/2, time_units='ms')
ax = first_pulse_traces.plot(plot_voltage=False, plot_ttl=False, time_units='ms', sweep='all')
ax.set_title('First paired-pulse stim',y=0.95)
plt.show()

#### Optional: remove bad traces

In [ ]:
remove_spikes = False

# ---------------------------------------------------------------------------------------------------------
if remove_spikes:
    # Remove sweeps with positive current spikes above or below threshold
    info = first_pulse_traces.remove_sweeps_with_spikes(threshold=-2000, polarity='negative', 
                                                channel='current', return_info=True)
    ax = first_pulse_traces.plot(plot_voltage=False, plot_ttl=False, time_units='ms', sweep='all')
    # ax.set_ylim(bottom=-600, top=100)
    ax.set_title('First paired-pulse stim (spikes removed)',y=0.95)
    plt.show()

#### Measure first paired-pulse amplitudes

In [ ]:
first_pulse_currents,_ = first_pulse_traces.get_measurements(start_time=4, 
                                                            end_time=15, 
                                                            measurement_type='min', 
                                                            time_units='ms')
fig,ax = plt.subplots(figsize=(8,3))
ax.plot(first_pulse_currents, '-o')
ax.set_xticks(range(len(first_pulse_currents)))
ax.set_ylabel('first paired-pulse currents (pA)')
ax.set_xlabel("sweep nr.")
plt.show()

### 3.3. Get the response to the second paired-pulse stim

#### Subtract single-stim response from paired-pulse trace

This allows us to recover the shape/amplitude of the second pulse, without the contribution of the first pulse

In [ ]:
paired_pulse_traces = traces.crop(timepoint=stim_times[1], window=window_size, time_units='ms')
ax = paired_pulse_traces.plot(plot_voltage=False, plot_ttl=False, time_units='ms', sweep='all')
ax.set_title('Paired stim',y=0.95)
plt.show()

Now we will get the average "first stim" response and subtract it from all of the paired-pulse sweeps

In [ ]:
subtracted_traces = paired_pulse_traces.copy()
subtract_average_single_stim = True   # We can either subtract the average single-stim, or subtract each individual single-stim (from every sweep)

if subtract_average_single_stim:
    avg_single_stim_trace = np.mean(single_stim_traces.current_data, axis=0)
    subtracted_traces.current_data -= avg_single_stim_trace
else:
    subtracted_traces.current_data -= single_stim_traces.current_data

ax = subtracted_traces.plot(plot_voltage=False, plot_ttl=False, time_units='ms', sweep='all')
ax.set_title('After subtraction',y=0.95)
plt.show()

#### Measure second paired-pulse amplitudes

In [ ]:
# Find stim artifact of the subtracted trace
stim_times = subtracted_traces.get_event_times(threshold=200, 
                                            polarity='positive', 
                                            time_units='ms', 
                                            channel='current', sweep=0)
print(f"Stim times: {stim_times} ms")

In [ ]:
second_pulse_currents,_ = subtracted_traces.get_measurements(start_time=stim_times[0]+2, 
                                                            end_time=stim_times[0]+17, 
                                                            measurement_type='min', 
                                                            time_units='ms')
fig,ax = plt.subplots(figsize=(8,3))
ax.plot(second_pulse_currents, '-o')
ax.set_xticks(range(len(second_pulse_currents)))
ax.set_ylabel('second paired-pulse currents (pA)')
ax.set_xlabel("sweep nr.")
plt.show()

### 3.4. Plot paired-pulse ratio

In [ ]:
if len(first_pulse_currents) == len(second_pulse_currents):
    paired_pulse_ratios = second_pulse_currents / first_pulse_currents
    avg_ppr = np.mean(paired_pulse_ratios)
    fig,ax = plt.subplots()
    ax.plot(paired_pulse_ratios, '-o')
    ax.set_xticks(range(len(second_pulse_currents)))
    xlims = ax.get_xlim()
    ax.set_ylim(bottom=0)
    ylims = ax.get_ylim()
    ax.hlines(1, xlims[0],xlims[1],linestyle='--',color='gray')
    ax.text(0, ylims[1], f"Paired-pulse ratio: {avg_ppr:.2f}", fontsize=10, ha='left', va='bottom')
    ax.set_ylabel('Paired-pulse ratios')
    ax.set_xlabel("sweep nr.")
    plt.show()
else:
    avg_second_pulse = np.mean(second_pulse_currents)
    avg_first_pulse = np.mean(first_pulse_currents)
    paired_pulse_ratio = avg_second_pulse / avg_first_pulse
    print(f"Paired-pulse ratio: {paired_pulse_ratio}")
